In [ ]:
# import Biopython functions
from Bio import SeqIO

# Loading and viewing sequences
In this example both a transcript (Thy1 in mouse), and the coding sequence, are read.

In [ ]:
# load the whole transcipt
transcript = next(SeqIO.parse("complete.fasta", "fasta"))

# load the coding sequence
coding = next(SeqIO.parse("coding.fasta", "fasta"))

In [ ]:
print(transcript.description)

In [ ]:
print(coding.description)

In [ ]:
print(transcript.seq)

In [ ]:
print(coding.seq)

In [ ]:
print("Lenth of gene:",len(transcript.seq))
print("Lenth of coding sequence:",len(coding.seq))

In [ ]:
# find the start of the coding sequence in the gene
# we know from the info above that the gene starts at location 977
# python slicing offers quick access, note that python starts counting at zero:
transcript.seq[976:]

# Simple sequence operations 

In [ ]:
transcript.seq.count('A')

## Two ways to compute GC content

G-C pairs have a stronger bonds due to an addition H-bond
<img src="GC.png">

In [ ]:
print(100 * float(transcript.seq.count("G") + transcript.seq.count("C")) / len(transcript.seq))

from Bio.SeqUtils import GC
print(GC(transcript.seq))

## find first occurence of a substring

In [ ]:
print(transcript.seq.find('ATG'))
print(coding.seq.find('ATG'))

## obtain the complemenatry sequence
compare with sequence above to see whether it makes sense to you

In [ ]:
print(transcript.seq.complement())

# Transcription and translation

In [ ]:
print(coding.seq.transcribe())

In [ ]:
print(coding.seq.translate())

We could also tranlate the whole RNA:

In [ ]:
print(transcript.seq.translate())

This sequence is different from the translated coding sequence, and does not even contain a subset of the sequence above. And, we get a warning. Why?

An alternative is to start at the beginning of the coding sequence (see above):

In [ ]:
print(transcript.seq[976:].translate())

This gives a sensible result for our example. However, tge sequence is longer than the one we had above for the coding sequance. Does this matter? Find an example where it does not work. Why? 

We could also search for a start codon:

In [ ]:
codingStart = transcript.seq.find('ATG')
print(codingStart)
print(transcript.seq[codingStart:].translate())

Does this make sense?

# Open Reading Frames (ORFs)

How many start codons are there? Where are they?

In [ ]:
print(transcript.seq.count('ATG'))

In [ ]:
seq = transcript.seq
i = 0
while seq[i:].find('ATG')>-1:
    j = seq[i:].find('ATG')
    i = i + j + 3
    print(i - 3)


This is a list of possible coding region starting points.

## Alternative strategy

Each true ORF should end with a stop codon. So we can translate the sequence using all three different reading farmes, and search for start-> stop codon sequences.

In [ ]:
f1 = transcript.seq.translate().split('*')
f2 = transcript.seq[1:].translate().split('*')
f3 = transcript.seq[2:].translate().split('*')
print(f1)


These are all candidate coding sequences ending with a stop codon. Now check if there is a start codon. Since we have now got amino acis sequences, we should search for the amino acid equivalent of 'ATG', which is 'M' (Methionine)

In [ ]:
for i,f in enumerate((f1,f2,f3)):
    print("Reading frame: "+str(i+1))
    for p in f:
        M = p.find('M')
        if M>=0:
            print((len(p)-M+1)*3)

These are the lengths of all possible open reading frames. There are many different candidates. Often the longest is the correct one, here that is the case too.

Note: here we know the 5' -> 3' direction (can you see why?), but generally both directions should be searched.

## Putting everything together

In [ ]:
# a simple ORF finder

seq = transcript.seq

# search for at least 100 amino acid length:
min_len = 100

for strand, nuc in [(+1, seq), (-1, seq.reverse_complement())]:
    for frame in range(3):
        pos = 0
        for prot in nuc[frame:].translate().split('*'):
            if len(prot) >= min_len:
                start = prot.find("M")
                print("%s...%s - fragment length: %i, strand: %i, frame: %i, pos: %i, start: %i, coding length %i, has start %s" % (prot[start:start+10], prot[-3:], (len(prot)-start+1)*3, strand, frame, pos*3, (max(0,start)+pos+frame)*3+1, (len(prot[start:])+1)*3, start>-1))
                pos = pos + len(prot)+1